In [1]:
import itk
import dask.array as da
import dask
from dask.array.image import imread
from dask.distributed import Client, progress, LocalCluster
import dask.array as da
client = Client(n_workers=8)#, serializers=['pickle'])#, processes=False)
client

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44707 instead


Client Scheduler: tcp://127.0.0.1:40961 Dashboard: http://127.0.0.1:44707/status,Cluster Workers: 8 Cores: 8 Memory: 16.63 GB


In [2]:
def load_image(path):
    itk_img = itk.imread(path)
    image_np = itk.array_from_image(itk_img)
    return itk_img 
itk_img = load_image("data/test_data.tif")

In [3]:
template_type = type(itk_img)
from distributed.protocol import register_generic, dask_serialize, dask_deserialize
from distributed.protocol.serialize import dask_dumps
from typing import Any, Dict, Tuple, List, Union

@dask_serialize.register(template_type) 
def serialize(itk_image: template_type) -> Tuple[Dict, List[bytes]]:
#     import pdb; pdb.set_trace()
    import itk
    np_array = itk.array_from_image(itk_image)
    
    ret = dask_dumps(np_array)
    return ret
import numpy as np
@dask_deserialize.register(template_type)
def deserialize(header: Dict, frames: List[bytes]) -> template_type:
    import pdb; pdb.set_trace()
    import itk
    loads = dask_deserialize.dispatch(np.ndarray)
    return itk.image_from_array(loads(header, frames))

In [4]:
type(itk_img)

itk.itkImagePython.itkImageUC3

In [5]:
#chunked_data  = da.from_array(np_img, chunks=(200, 200, 5))  ## 40 blocks
#%debug
scattered_data = client.scatter(itk_img, broadcast=True)

distributed.core - ERROR - No dispatch for <class 'itk.itkImagePython.itkImageUC3'>
Traceback (most recent call last):
  File "/home/zhiyuan.liu/anaconda2/envs/test_serialize/lib/python3.7/site-packages/distributed/core.py", line 513, in handle_comm
    result = await result
  File "/home/zhiyuan.liu/anaconda2/envs/test_serialize/lib/python3.7/site-packages/distributed/scheduler.py", line 2839, in scatter
    nthreads, data, rpc=self.rpc, report=False
  File "/home/zhiyuan.liu/anaconda2/envs/test_serialize/lib/python3.7/site-packages/distributed/utils_comm.py", line 149, in scatter_to_workers
    for address, v in d.items()
  File "/home/zhiyuan.liu/anaconda2/envs/test_serialize/lib/python3.7/site-packages/distributed/utils.py", line 229, in All
    result = await tasks.next()
  File "/home/zhiyuan.liu/anaconda2/envs/test_serialize/lib/python3.7/site-packages/distributed/core.py", line 862, in send_recv_from_rpc
    result = await send_recv(comm=comm, op=key, **kwargs)
  File "/home/zh

Exception: No dispatch for <class 'itk.itkImagePython.itkImageUC3'>

In [8]:
scattered_data.result()

,Array,Chunk
Bytes,5.24 MB,20.00 kB
Shape,"(20, 512, 512)","(20, 200, 5)"
Count,310 Tasks,309 Chunks
Type,uint8,numpy.ndarray


In [9]:
denoised = client.submit(denoise, scattered_data).result()
type(denoised)

distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/zhiyuan.liu/anaconda2/envs/test_serialize/lib/python3.7/site-packages/distributed/protocol/core.py", line 130, in loads
    value = _deserialize(head, fs, deserializers=deserializers)
  File "/home/zhiyuan.liu/anaconda2/envs/test_serialize/lib/python3.7/site-packages/distributed/protocol/serialize.py", line 304, in deserialize
    return loads(header, frames)
  File "/home/zhiyuan.liu/anaconda2/envs/test_serialize/lib/python3.7/site-packages/distributed/protocol/serialize.py", line 52, in dask_loads
    loads = dask_deserialize.dispatch(typ)
  File "/home/zhiyuan.liu/anaconda2/envs/test_serialize/lib/python3.7/site-packages/dask/utils.py", line 500, in dispatch
    raise TypeError("No dispatch for {0}".format(cls))
TypeError: No dispatch for <class 'itk.itkPyBufferPython.NDArrayITKBase'>


TypeError: No dispatch for <class 'itk.itkPyBufferPython.NDArrayITKBase'>